In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ax.service.managed_loop import optimize
from statistics import mean
from pykalman import KalmanFilter

import logging
from ax.utils.common.logger import get_logger

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

# Only show log messages of ERROR while testing.
logger = get_logger(__name__, level=logging.ERROR)
if logger.parent is not None and hasattr(logger.parent, "handlers"):
    logger.parent.handlers[0].setLevel(logging.ERROR)

# Load and preprocess data
data = pd.read_csv('OCULUS_dataset.csv')
data.fillna(data.mean(), inplace=True)

features = data.drop(["game_section", "stress_label", "subject"], axis=1)
labels = data["stress_label"]

scaler = StandardScaler()
features = scaler.fit_transform(features)

features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
labels_tensor = torch.tensor(labels.values, dtype=torch.float32).to(device)

class StressLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate_1, dropout_rate_2, dropout_rate_fc):
        super(StressLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate_1)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate_2)
        self.dropout = nn.Dropout(dropout_rate_fc)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out = self.fc(out).squeeze(-1)
        return out

parameters = [
    {"name": "lr", "type": "range", "bounds": [1e-6, 1e-3], "log_scale": True, "value_type": "float"},
    {"name": "hidden_size", "type": "range", "bounds": [50, 256], "value_type": "int"},
    {"name": "num_layers", "type": "range", "bounds": [2, 3], "value_type": "int"},
    {"name": "num_epochs", "type": "fixed", "value": 150, "value_type": "int"},
    {"name": "dropout_rate_1", "type": "range", "bounds": [0.0, 0.7], "value_type": "float"},
    {"name": "dropout_rate_2", "type": "range", "bounds": [0.0, 0.7], "value_type": "float"},
    {"name": "dropout_rate_fc", "type": "range", "bounds": [0.0, 0.7], "value_type": "float"},
    {"name": "weight_decay", "type": "range", "bounds": [1e-6, 1], "value_type": "float"},
    {"name": "batch_size", "type": "range", "bounds": [10, 100], "value_type": "int"}
]

criterion = nn.MSELoss().to(device)
mae_criterion = nn.L1Loss().to(device)

def evaluate_full_dataset(parameters, features, labels):
    print("Evaluating with parameters: " + str(parameters))

    # Split data
    X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, shuffle=False)

    # Create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_dataloader = DataLoader(train_dataset, batch_size=int(parameters['batch_size']), shuffle=False)

    # Model initialization
    model = StressLSTM(input_size=features.shape[1],
                       hidden_size=int(parameters['hidden_size']),
                       num_layers=int(parameters['num_layers']),
                       dropout_rate_1=parameters['dropout_rate_1'],
                       dropout_rate_2=parameters['dropout_rate_2'],
                       dropout_rate_fc=parameters['dropout_rate_fc']).to(device)

    # Optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=parameters['lr'], weight_decay=parameters['weight_decay'])
    
    best_val_loss = float('inf')
    patience, patience_threshold = 0, 10  # Early stopping

    for epoch in range(int(parameters['num_epochs'])):
        model.train()
        running_loss = 0.0

        # Training loop
        for batch_features, batch_labels in train_dataloader:
            batch_features, batch_labels = batch_features.unsqueeze(0).to(device), batch_labels.view(-1).to(device)

            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val.unsqueeze(0).to(device))
            val_loss = criterion(val_outputs.view(-1), y_val.view(-1).to(device))

        print(f'Epoch [{epoch+1}/{int(parameters["num_epochs"])}], Loss: {running_loss/len(train_dataloader)}, Val Loss: {val_loss.item()}')

        # Early stopping logic
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience = 0
        else:
            patience += 1
            if patience >= patience_threshold:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return best_val_loss

def evaluate_with_objective(parameters):
    mse_loss = evaluate_full_dataset(parameters, features_tensor, labels_tensor)
    print(f"MSE Loss: {mse_loss}")
    return mse_loss  # Return only MSE as the optimization target


best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=evaluate_with_objective,
    objective_name='loss'
)

print(best_parameters)

input_size = features.shape[1]
output_size = 1
learning_rate = best_parameters['lr']
hidden_size = int(best_parameters['hidden_size'])
num_layers = int(best_parameters['num_layers'])
num_epochs = int(best_parameters['num_epochs'])
dropout_rate_1 = best_parameters['dropout_rate_1']
dropout_rate_2 = best_parameters['dropout_rate_2']
dropout_rate_fc = best_parameters['dropout_rate_fc']
weight_decay = best_parameters['weight_decay']
batch_size = int(best_parameters['batch_size'])

tscv = TimeSeriesSplit(n_splits=5)
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

def calculate_loss_and_mae(model, criterion, mae_criterion, inputs, targets, use_kalman_filter=False):
    outputs = model(inputs.unsqueeze(0).to(device))
    if use_kalman_filter:
        (filtered_state_means, filtered_state_covariances) = kf.filter(outputs.cpu().detach().numpy())
        outputs = torch.tensor(filtered_state_means, dtype=torch.float32).view(1, -1).to(device)
    loss = criterion(outputs, targets.unsqueeze(0).to(device))
    mae = mae_criterion(outputs, targets.unsqueeze(0).to(device))
    return loss, mae

grouped = data.groupby('subject')

def do_stuff():
    all_X_train, all_y_train, all_X_val, all_y_val, all_X_test, all_y_test = [], [], [], [], [], []
    for name, group in grouped:
        print(f"Subject: {name}")
        features = group.drop(["game_section", "stress_label", "subject"], axis=1)
        labels = group["stress_label"]
    
        scaler = StandardScaler()
        features = scaler.fit_transform(features)
    
        features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
        labels_tensor = torch.tensor(labels.values, dtype=torch.float32).to(device)
    
        for train_index, test_index in tscv.split(features_tensor):
            X_train_val, X_test = features_tensor[train_index], features_tensor[test_index]
            y_train_val, y_test = labels_tensor[train_index], labels_tensor[test_index]
    
            X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=False)
    
            all_X_train.append(X_train)
            all_y_train.append(y_train)
            all_X_val.append(X_val)
            all_y_val.append(y_val)
            all_X_test.append(X_test)
            all_y_test.append(y_test)
    
    X_train = torch.cat(all_X_train).to(device)
    y_train = torch.cat(all_y_train).to(device)
    X_val = torch.cat(all_X_val).to(device)
    y_val = torch.cat(all_y_val).to(device)
    X_test = torch.cat(all_X_test).to(device)
    y_test = torch.cat(all_y_test).to(device)
    
    window_sizes = [2, 4, 6]
    
    all_train_losses, all_val_losses, all_train_maes, all_val_maes, losses, maes = [], [], [], [], [], []
    all_filtered_train_losses, all_filtered_val_losses, all_filtered_train_maes, all_filtered_val_maes, filtered_losses, filtered_maes = [], [], [], [], [], []
    
    for window_size in window_sizes:
        print(f"Window Size: {window_size}")
        if len(X_train) < window_size or len(y_train) < window_size:
            continue
    
        train_dataset = TensorDataset(X_train[:len(X_train)//window_size*window_size], y_train[:len(y_train)//window_size*window_size])
        train_dataloader = DataLoader(train_dataset, batch_size=window_size, shuffle=False)
    
        model = StressLSTM(input_size, hidden_size, num_layers, dropout_rate_1, dropout_rate_2, dropout_rate_fc).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
        model.eval()
        with torch.no_grad():
            training_outputs = model(X_train.unsqueeze(0).to(device))
            kf.em(training_outputs.cpu().detach().numpy(), n_iter=10)
    
        train_losses, val_losses, train_maes, val_maes = [], [], [], []
        filtered_train_losses, filtered_val_losses, filtered_train_maes, filtered_val_maes = [], [], [], []
        for epoch in range(num_epochs):
            print(f"Epoch {epoch+1}/{num_epochs}", end="\r")
            running_loss = 0.0
            running_mae = 0.0
            running_filtered_loss = 0.0
            running_filtered_mae = 0.0
            num_batches = 0
    
            for batch_features, batch_labels in train_dataloader:
                model.train()
                loss, mae = calculate_loss_and_mae(model, criterion, mae_criterion, batch_features, batch_labels)
                filtered_loss, filtered_mae = calculate_loss_and_mae(model, criterion, mae_criterion, batch_features, batch_labels, use_kalman_filter=True)
    
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    
                running_loss += loss.item()
                running_mae += mae.item()
                running_filtered_loss += filtered_loss.item()
                running_filtered_mae += filtered_mae.item()
                num_batches += 1
    
            average_train_loss = running_loss / num_batches
            average_train_mae = running_mae / num_batches
            average_filtered_loss = running_filtered_loss / num_batches
            average_filtered_mae = running_filtered_mae / num_batches
            train_losses.append(average_train_loss)
            train_maes.append(average_train_mae)
            filtered_train_losses.append(average_filtered_loss)
            filtered_train_maes.append(average_filtered_mae)
    
            model.eval()
            with torch.no_grad():
                val_loss, val_mae = calculate_loss_and_mae(model, criterion, mae_criterion, X_val, y_val)
                val_filtered_loss, val_filtered_mae = calculate_loss_and_mae(model, criterion, mae_criterion, X_val, y_val, use_kalman_filter=True)
    
            val_losses.append(val_loss.item())
            val_maes.append(val_mae.item())
            filtered_val_losses.append(val_filtered_loss.item())
            filtered_val_maes.append(val_filtered_mae.item())
    
            model.eval()
            with torch.no_grad():
                test_loss, test_mae = calculate_loss_and_mae(model, criterion, mae_criterion, X_test, y_test)
                test_filtered_loss, test_filtered_mae = calculate_loss_and_mae(model, criterion, mae_criterion, X_test, y_test, use_kalman_filter=True)
    
            losses.append(test_loss.item())
            maes.append(test_mae.item())
            filtered_losses.append(test_filtered_loss.item())
            filtered_maes.append(test_filtered_mae.item())
    
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))
        
        maxy = max(max(train_losses), max(val_losses), max(filtered_train_losses), max(filtered_val_losses), max(train_maes), max(val_maes), max(filtered_train_maes), max(filtered_val_maes))
    
        axs[0, 0].plot(train_losses, label='Training MSE')
        axs[0, 0].plot(val_losses, label='Validation MSE')
        axs[0, 0].set_xlabel('Epoch')
        axs[0, 0].set_ylabel('MSE')
        axs[0, 0].set_ylim(0, maxy)
        axs[0, 0].legend()
        axs[0, 0].set_title('Training and Validation MSE')
    
        axs[0, 1].plot(filtered_train_losses, label='Filtered Training MSE')
        axs[0, 1].plot(filtered_val_losses, label='Filtered Validation MSE')
        axs[0, 1].set_xlabel('Epoch')
        axs[0, 1].set_ylabel('MSE')
        axs[0, 1].set_ylim(0, maxy)
        axs[0, 1].legend()
        axs[0, 1].set_title('Filtered Training and Validation MSE')
    
        axs[1, 0].plot(train_maes, label='Training MAE')
        axs[1, 0].plot(val_maes, label='Validation MAE')
        axs[1, 0].set_xlabel('Epoch')
        axs[1, 0].set_ylabel('MAE')
        axs[1, 0].set_ylim(0, maxy)
        axs[1, 0].legend()
        axs[1, 0].set_title('Training and Validation MAE')
    
        axs[1, 1].plot(filtered_train_maes, label='Filtered Training MAE')
        axs[1, 1].plot(filtered_val_maes, label='Filtered Validation MAE')
        axs[1, 1].set_xlabel('Epoch')
        axs[1, 1].set_ylabel('MAE')
        axs[1, 1].set_ylim(0, maxy)
        axs[1, 1].legend()
        axs[1, 1].set_title('Filtered Training and Validation MAE')
    
        fig.suptitle(f'Window Size: {window_size}')
        plt.tight_layout()
        plt.show()
    
        print("Average test MSE :", mean(losses))
        print("Average test MAE :", mean(maes))
    
        print("Average filtered test MSE :", mean(filtered_losses))
        print("Average filtered test MAE :", mean(filtered_maes))
        
for i in range(10):
    do_stuff()

Using device:  cuda


C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\ax\modelbridge\cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))


Evaluating with parameters: {'lr': 1.7200460023147145e-06, 'hidden_size': 246, 'num_layers': 3, 'dropout_rate_1': 0.5831521153450012, 'dropout_rate_2': 0.6510676383972167, 'dropout_rate_fc': 0.36499016284942626, 'weight_decay': 0.43428298778331276, 'batch_size': 85, 'num_epochs': 150}


C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([85])) that is different to the input size (torch.Size([1, 85])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([1, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/150], Loss: 0.38639323388115837, Val Loss: 0.2935720980167389
Epoch [2/150], Loss: 0.38563668489519315, Val Loss: 0.2930510342121124
Epoch [3/150], Loss: 0.38521109109397156, Val Loss: 0.2925291955471039
Epoch [4/150], Loss: 0.384090798417643, Val Loss: 0.2920148968696594
Epoch [5/150], Loss: 0.38378547341957436, Val Loss: 0.2915027141571045
Epoch [6/150], Loss: 0.38269740917672546, Val Loss: 0.29099616408348083
Epoch [7/150], Loss: 0.3818275301196313, Val Loss: 0.29049786925315857
Epoch [8/150], Loss: 0.38060208078536945, Val Loss: 0.2900025546550751
Epoch [9/150], Loss: 0.38003199320223374, Val Loss: 0.28951606154441833
Epoch [10/150], Loss: 0.37940295454177814, Val Loss: 0.28903499245643616
Epoch [11/150], Loss: 0.3782068853092901, Val Loss: 0.28855812549591064
Epoch [12/150], Loss: 0.3776259412673318, Val Loss: 0.2880879044532776
Epoch [13/150], Loss: 0.37703609996933046, Val Loss: 0.28762325644493103
Epoch [14/150], Loss: 0.37612651260096136, Val Loss: 0.28717005252838135

...

Epoch [1/150], Loss: 0.3192296041739023, Val Loss: 0.24088649451732635
Epoch [2/150], Loss: 0.2750682629119124, Val Loss: 0.22954612970352173
Epoch [3/150], Loss: 0.2510419258246979, Val Loss: 0.22605782747268677
Epoch [4/150], Loss: 0.23709410199174477, Val Loss: 0.2257470339536667
Epoch [5/150], Loss: 0.228907853111835, Val Loss: 0.22651952505111694
Epoch [6/150], Loss: 0.22419257704468984, Val Loss: 0.2275024801492691
Epoch [7/150], Loss: 0.22108845101939814, Val Loss: 0.22838759422302246
Epoch [8/150], Loss: 0.21890989829065097, Val Loss: 0.22908934950828552
Epoch [9/150], Loss: 0.21782379559121065, Val Loss: 0.22959372401237488
Epoch [10/150], Loss: 0.21709655755830387, Val Loss: 0.22996830940246582
Epoch [11/150], Loss: 0.2162699914972989, Val Loss: 0.23023676872253418
Epoch [12/150], Loss: 0.2157152942471927, Val Loss: 0.23045334219932556
Epoch [13/150], Loss: 0.21561526986019267, Val Loss: 0.2305585741996765
Epoch [14/150], Loss: 0.21538227174671426, Val Loss: 0.230671390891075

C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\ax\modelbridge\cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([73])) that is different to the input size (torch.Size([1, 73])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([67])) that is different to the input size (torch.Size([1, 67])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduc

Epoch [1/150], Loss: 0.43938867727239783, Val Loss: 0.3245663046836853
Epoch [2/150], Loss: 0.43873702749089505, Val Loss: 0.324272483587265
Epoch [3/150], Loss: 0.4378850019797135, Val Loss: 0.3239773213863373
Epoch [4/150], Loss: 0.43682836888440174, Val Loss: 0.323682963848114
Epoch [5/150], Loss: 0.43675117351470844, Val Loss: 0.3233848214149475
Epoch [6/150], Loss: 0.4359564622179331, Val Loss: 0.3230940103530884
Epoch [7/150], Loss: 0.43637115754844513, Val Loss: 0.32280170917510986
Epoch [8/150], Loss: 0.43600217673434494, Val Loss: 0.3225134313106537
Epoch [9/150], Loss: 0.43492259199539307, Val Loss: 0.3222251236438751
Epoch [10/150], Loss: 0.4343405004371615, Val Loss: 0.32193970680236816
Epoch [11/150], Loss: 0.4347143950050368, Val Loss: 0.321659117937088
Epoch [12/150], Loss: 0.4333549762651434, Val Loss: 0.3213786482810974
Epoch [13/150], Loss: 0.43372963564744327, Val Loss: 0.3210982382297516
Epoch [14/150], Loss: 0.4330263484872001, Val Loss: 0.32082244753837585
Epoch [

C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([83])) that is different to the input size (torch.Size([1, 83])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Marco Ligabue\.conda\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([61])) that is different to the input size (torch.Size([1, 61])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/150], Loss: 0.36198062244802715, Val Loss: 0.27820777893066406
Epoch [2/150], Loss: 0.3615946862846613, Val Loss: 0.2780826687812805
Epoch [3/150], Loss: 0.36221623203406733, Val Loss: 0.2779569923877716
Epoch [4/150], Loss: 0.3615015803525845, Val Loss: 0.2778315544128418
Epoch [5/150], Loss: 0.36167225607981285, Val Loss: 0.27770593762397766
Epoch [6/150], Loss: 0.3610603898142775, Val Loss: 0.27758118510246277
Epoch [7/150], Loss: 0.36078667544449367, Val Loss: 0.27745571732521057
Epoch [8/150], Loss: 0.3613309249592324, Val Loss: 0.27733203768730164
Epoch [9/150], Loss: 0.3608045004929105, Val Loss: 0.2772078812122345
Epoch [10/150], Loss: 0.36001117223252854, Val Loss: 0.2770856022834778
Epoch [11/150], Loss: 0.3599764894073208, Val Loss: 0.2769635021686554
Epoch [12/150], Loss: 0.3600929093857606, Val Loss: 0.27684152126312256
Epoch [13/150], Loss: 0.35972173636158306, Val Loss: 0.27672079205513
Epoch [14/150], Loss: 0.3595679118918876, Val Loss: 0.27659985423088074
Epoc